# NoteBook Ingest dataset export_goods_services_dataset

***Author:*** Frederick salazar <br>
***Data Source:*** https://datos.bancomundial.org/indicador/NE.EXP.GNFS.ZS?view=chart <br>
***Description:*** This dataset contains data on exports of goods and services as a percentage of GDP for each country from 1960 to 2023. The data is obtained from the World Bank repository and exported in tabular form in CSV format.

## Importations and configurations

In [315]:
#library importations

import pandas as pd

import os
import requests
import zipfile
from io import BytesIO
from unidecode import unidecode

version = 1.0

In [316]:
#Path configurations

raw = './data/raw'
processed_data = './data/processed'
output_data = './data/output'

output_dataset = './data/processed/export_goods_services_countries_dataset.csv'

In [317]:
def donwload_data(data_source_in):
    """this function get files from url and save it in the raw folder

    Args:
        data_source_in (String): url of the file to download
    """
    response = requests.get(data_source_in)

    if response.status_code == 200:
        # Leer el contenido del archivo ZIP en memoria
        zip_file = BytesIO(response.content)
        
        # Descomprimir el contenido del ZIP directamente en el directorio
        with zipfile.ZipFile(zip_file) as z:
            z.extractall(raw)

## Download and read data

In [318]:
#here use the function to donwload data from url
donwload_data('https://api.worldbank.org/v2/es/indicator/NE.EXP.GNFS.ZS?downloadformat=csv')

## load data

In [319]:
for file in os.listdir(raw):
    print(file)
    #load data of exports
    if file.startswith('API_NE.EXP.GNFS.ZS_DS2_es_csv_v2_5037'):
        df_exports = pd.read_csv(f'{raw}/{file}',
                                sep=',',
                                skiprows=4)
        df_exports = df_exports.iloc[:, :-1]

    #load metadata of countries
    if file.startswith('Metadata_Country'):
        df_metadata = pd.read_csv(f'{raw}/{file}',
                                    sep=',',
                                    skiprows=0)
        df_metadata = df_metadata.iloc[:, :-1]
    
    #load metadata of country codes
    if file.startswith('country_codes_list'):
        df_country_code = pd.read_csv(f'{raw}/{file}',
                                    sep=',',
                                    skiprows=0)

Metadata_Country_API_NE.EXP.GNFS.ZS_DS2_es_csv_v2_5037.csv
.gitkeep
API_NE.EXP.GNFS.ZS_DS2_es_csv_v2_5037.csv
Metadata_Indicator_API_NE.EXP.GNFS.ZS_DS2_es_csv_v2_5037.csv
country_codes_list.csv


## Transform data

### Select Columns

In [320]:
df_country_code = df_country_code[['alpha-3',
                                   'sub-region',
                                   'intermediate-region']]

### Format and columns transformations

In [321]:
#convertimos todos los strings en upper
df_exports = df_exports.applymap(lambda x: x.upper() if type(x) == str else x)
df_metadata = df_metadata.applymap(lambda x: x.upper() if type(x) == str else x)
df_country_code = df_country_code.applymap(lambda x: x.upper() if type(x) == str else x)

#delete acents
df_exports = df_exports.applymap(lambda x: unidecode(x) if type(x) == str else x)

/var/folders/34/mqqssx0s7jd53nv2vzj0plc00000gn/T/ipykernel_26975/489681352.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_exports = df_exports.applymap(lambda x: x.upper() if type(x) == str else x)
/var/folders/34/mqqssx0s7jd53nv2vzj0plc00000gn/T/ipykernel_26975/489681352.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_metadata = df_metadata.applymap(lambda x: x.upper() if type(x) == str else x)
/var/folders/34/mqqssx0s7jd53nv2vzj0plc00000gn/T/ipykernel_26975/489681352.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_country_code = df_country_code.applymap(lambda x: x.upper() if type(x) == str else x)
/var/folders/34/mqqssx0s7jd53nv2vzj0plc00000gn/T/ipykernel_26975/489681352.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_exports = df_exports.applymap(lambda x: unidecode(x) if type(x) == str else x)


### Structure transformation

In [322]:
#change from large table to long table
df_exports = pd.melt(df_exports,
                     id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
                     var_name='year',
                     value_name='exports_of_goods_and_services')

In [323]:
#change de column name
df_exports.rename(columns={'Country Name': 'country_name',
                           'Country Code': 'country_code',
                           'Indicator Name': 'indicator_name',
                           'Indicator Code': 'indicator_code'},
                  inplace=True)

df_metadata.rename(columns={'Country Name': 'country_name',
                            'Country Code': 'country_code',
                            'Region': 'region',
                            'Income_Group': 'income_group'},
                   inplace=True)

df_country_code.rename(columns={'alpha-3': 'alpha_3',
                                'sub-region': 'sub_region',
                                'intermediate-region': 'intermediate_region'},
                       inplace=True)

In [324]:
df_exports['exports_of_goods_and_services'] = df_exports['exports_of_goods_and_services'].fillna(0)

### Merge of dataframes

In [325]:
df_exports = pd.merge(df_exports, 
                      df_metadata, 
                      on='country_code', how='left')

In [326]:
df_exports = df_exports[['country_name_x',
                         'country_code',
                         'region',
                         'income_group',
                         'indicator_name',
                         'indicator_code',
                         'year',
                         'exports_of_goods_and_services']]

df_exports.rename(columns={'country_name_x': 'country_name'}, inplace=True)

In [327]:
#get only countries
df_exports_countries = pd.merge(df_exports,
                                df_country_code,
                                left_on='country_code',
                                right_on='alpha_3',
                                how='inner').drop(columns=['alpha_3'])

#get only organizations exports data
df_exports_organizations = df_exports[~df_exports['country_code'].isin(df_country_code['alpha_3'])]

df_exports_countries = df_exports_countries[['country_code',
                                             'country_name',
                                             'region',
                                             'sub_region',
                                             'intermediate_region',
                                             'indicator_code',
                                             'indicator_name',
                                             'year',
                                             'exports_of_goods_and_services']]

In [328]:
df_exports_countries.sort_values(by=['country_name', 'year'], inplace=True)
df_exports_organizations.sort_values(by=['country_name', 'year'], inplace=True)

/var/folders/34/mqqssx0s7jd53nv2vzj0plc00000gn/T/ipykernel_26975/450851505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_exports_organizations.sort_values(by=['country_name', 'year'], inplace=True)


## Save processed data

In [329]:
df_exports_countries.to_csv(output_dataset, index=False)
df_exports_organizations.to_csv(f'{processed_data}/export_goods_services_organizations.csv', index=False)